In [1]:
from IPython.core.display import display, HTML
import pandas as pd
from bs4 import BeautifulSoup
import requests
import os
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import sys
import numpy as np
import pandas as pd
import regex as re
import requests
import seaborn as sns
import lxml
from lxml.html.soupparser import fromstring
import numbers
import geopandas as gps
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
from sklearn.linear_model import LinearRegression


#%pip search geopy
import warnings
warnings.filterwarnings('ignore')

# Specify town/county

In [459]:
city = 'Larachmont-NY/'

## Webscrape both sold and listed homes for input area

In [460]:
req_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}

In [461]:
#ESTABLISH NUMBER OF PAGES IN THE SOLD SECTION 
with requests.Session() as s:
    url2 = 'https://www.zillow.com/'+city+'/sold/'
    url1=s.get(url2, headers=req_headers)
html1=BeautifulSoup(url1.content,"html.parser")

li_list=[i.text for i in html1.find_all('li')]

test=[j for j in li_list if "Page" in j]
pages=test[0]
pages2=re.search(r'\d+$', pages)
page_num_sold=int(pages2.group())if pages2 else None
print(page_num_sold)

20


In [462]:
#ESTABLISH NUMBER OF PAGES IN THE LISTING SECTION
with requests.Session() as s:
    url2 = 'https://www.zillow.com/'+city
    url1=s.get(url2, headers=req_headers)
html1=BeautifulSoup(url1.content,"html.parser")
    
li_list=[i.text for i in html1.find_all('li')]
test=[j for j in li_list if "Page" in j]
pages=test[0]
pages2=re.search(r'\d+$', pages)
page_num=int(pages2.group())if pages2 else None
print(page_num)

3


In [463]:
#ITERATE THROUGH THOSE PAGES FOR SOLD AND LISTED HOUSES, BUILDING INDVIDIUAL SOUP LISTS 
#num=range(page_num_sold)
num=range(30)
url_list=[]
for i in num:
    with requests.Session() as s:
        url = 'https://www.zillow.com/'+city+'/sold/'+str(i)+"_p/"
        url_list.append(s.get(url, headers=req_headers))
soup_list_sold=[]
for i in url_list:
    soup_list_sold.append(BeautifulSoup(i.content,"html.parser"))
    
###

#num2=range(page_num)
num=range(10)
url_list2=[]
for i in num2:
    with requests.Session() as s:
        url2 = 'https://www.zillow.com/'+city+str(i)+"_p/"
        url_list2.append(s.get(url2, headers=req_headers))
soup_list=[]
for i in url_list2:
    soup_list.append(BeautifulSoup(i.content,"html.parser"))


In [464]:
#CREATE DATAFRAMES FOR SOLD AND LISTED DATA FROM SOUPS 

#CREATE DATAFRAME FROM OUR SOLD DATA
df_sold=pd.DataFrame()
address_sold=[]
prices_sold=[]
beds_sold=[]
listings=[]
header_sold=[]
for soup in soup_list_sold:
    for i in soup:
        address_sold.extend(soup.find_all (class_= 'list-card-addr'))
        prices_sold.extend(soup.find_all (class_='list-card-price'))
        beds_sold.extend(soup.find_all("ul", class_="list-card-details"))
        header_sold.extend(soup.find_all(class_= "list-card-variable-text"))
        listings.extend(soup.find_all("a", class_="list-card-link"))
df_sold['price']=prices_sold
df_sold['address']=address_sold
df_sold['beds_data']=beds_sold
df_sold['header']=header_sold
df_sold.shape

home_links=[]
for i in listings: 
    home_links.append(i['href'])

unique_links=home_links[::2]

df_sold['links']=unique_links


###
#DATAFRAME FOR LISTED DATA 
df=pd.DataFrame()
address=[]
prices=[]
beds=[]
listings=[]
header=[]
for soup in soup_list:
    for i in soup:
        address.extend(soup.find_all (class_= 'list-card-addr'))
        prices.extend(soup.find_all (class_='list-card-price'))
        beds.extend(soup.find_all("ul", class_="list-card-details"))
        try:
            header.extend(soup.find_all(class_= "list-card-variable-text"))
        except:
            header.extend(np.nan)
        listings.extend(soup.find_all("a", class_="list-card-link"))
df['price']=prices
df['address']=address
df['beds_data']=beds
df['header']=header
#print(len(header))

home_links=[]
for i in listings: 
    home_links.append(i['href'])

unique_links=home_links[::2]

df['links']=unique_links

df_sold.head(5)

,price,address,beds_data,header,links
0,[$1.05M],"[63 Myrtle Blvd, Larchmont, NY 10538]","[[3, [ , , bds]], [3, [ , , ba]], [2,159, [ ...",[Sold 06/08/2021],https://www.zillow.com/homedetails/63-Myrtle-B...
1,[$3.60M],"[20 Linden Ave, Larchmont, NY 10538]","[[5, [ , , bds]], [6, [ , , ba]], [5,344, [ ...",[Sold 06/07/2021],https://www.zillow.com/homedetails/20-Linden-A...
2,[$1.62M],"[10 Cambridge Ct, Larchmont, NY 10538]","[[3, [ , , bds]], [4, [ , , ba]], [2,204, [ ...",[Sold 06/07/2021],https://www.zillow.com/homedetails/10-Cambridg...
3,[$1.15M],"[5 Birch Rd, Larchmont, NY 10538]","[[4, [ , , bds]], [2, [ , , ba]], [1,848, [ ...",[Sold 06/04/2021],https://www.zillow.com/homedetails/5-Birch-Rd-...
4,"[$261,000]","[212 Larchmont Acre W #2C, Larchmont, NY 10538]","[[2, [ , , bds]], [1, [ , , ba]], [900, [ , ...",[Sold 06/04/2021],https://www.zillow.com/homedetails/212-Larchmo...


## Clean dataframes 

In [465]:
#CLEAN DATA FOR SOLD AND LISTED DATAFRAMES 

#CLEAN SOLD DATA

beds_data5=[]
for i in df_sold['beds_data'].astype('str'):
    all_digi=[]
    all_digi = re.findall(r'\d+', i)
    beds_data5.append(all_digi)
df_sold['beds_data2']=beds_data5
lenn=[]
for i in df_sold['beds_data2']:
    lenn.append(len(i))
df_sold['len']=lenn
#df_sold['beds_data2']=df_sold['beds_data2'].astype(str)
#df_sold['len']=df_sold['beds_data2'].str.len()
drop_me=df_sold[df_sold['len']!=4].index
df_sold.drop(drop_me, inplace=True)


bed_data_num=[]
sqft_list=[]
for i in df_sold['beds_data'].astype('str'):
    bed_bath=[]
    all_digi = re.findall(r'\d+', i)
    bed_bath.extend(all_digi[0:2])
    sqft_num=[all_digi[-2],all_digi[-1]]
    sqft=str(all_digi[-2])+str(all_digi[-1])
    sqft_list.append(int(sqft))
    bed_data_num.append(bed_bath)      
df_sold['sqft']=sqft_list
df_sold['beds_data2']=bed_data_num
beds=[]
baths=[]
for i in df_sold['beds_data2']:
    beds.append(i[0])
    baths.append(i[1])
df_sold['beds']=beds
df_sold['baths']=baths
df_sold['beds'] = df_sold['beds'].astype(int)
df_sold['baths'] = df_sold['baths'].astype(int)

#CLEAN ADDRESS
df_sold['address'] = df_sold['address'].astype(str)
df_sold['address'] = df_sold['address'].replace('<address class="list-card-addr">', ' ', regex=True)
df_sold['address'] = df_sold['address'].replace('</address>', ' ', regex=True)

#CLEAN PRICE

df_sold['price']=df_sold['price'].astype(str)
df_sold['price'] = df_sold['price'].replace('<div class="list-card-price">', ' ', regex=True)
df_sold['price'] = df_sold['price'].replace('</div>', ' ', regex=True)
df_sold['price'] = df_sold['price'].str.replace(',', '',regex=True)
df_sold['price'] = df_sold['price'].str.replace('$', '',regex=True)
df_sold['price'] = df_sold['price'].str.replace(' ', '',regex=True)
clean_price=[]

df_sold['price']=df_sold['price'].astype(str)
for i in df_sold['price']:
    if "M" in i:
        a=float(i.replace("M",''))*1000000
        b=float(a)*1000000
        clean_price.append(a)
        #clean_price.append(float(str(i).replace("M",""))*1000000)
    else:
        clean_price.append(float(i))

df_sold['clean_price']=clean_price 
drop_me3=df_sold[df_sold['clean_price']<20000].index 
df_sold.drop(drop_me3, inplace=True) 

#print(float(str(df['price'][0:5]).replace("M",""))*1000000)


#CLEAN HEADER 
df_sold['header'] = df_sold['header'].astype(str)
df_sold['header'] = df_sold['header'].replace('<div class="list-card-variable-text list-card-img-overlay">', ' ', regex=True)
df_sold['header'] = df_sold['header'].replace('</div>', ' ', regex=True)

#ADD TOWNS COLUMN
towns=[]
for i in df_sold['address']:
    towns.append(i.split(',')[-2].rstrip(','))
df_sold['town']=towns



df_sold2=df_sold[['address','price','beds','baths','sqft','header','clean_price', 'town','links']]
df_sold3=df_sold2.drop_duplicates(subset ="address", inplace = False)

#________________________________________

#CLEAN BEDS, BATHS, SQFT OF OUR LISTED DATA
beds_data2=[]
for i in df['beds_data'].astype('str'):
    all_digi=[]
    all_digi = re.findall(r'\d+', i)
    beds_data2.append(all_digi)
df['beds_data2']=beds_data2
df['len']=df['beds_data2'].str.len()
drop_me=df[df['len']!=4].index
df.drop(drop_me, inplace=True)


bed_data_num=[]
sqft_list=[]
for i in df['beds_data'].astype('str'):
    bed_bath=[]
    all_digi = re.findall(r'\d+', i)
    bed_bath.extend(all_digi[0:2])
    sqft_num=[all_digi[-2],all_digi[-1]]
    sqft=str(all_digi[-2])+str(all_digi[-1])
    sqft_list.append(int(sqft))
    bed_data_num.append(bed_bath)      
df['sqft']=sqft_list
df['beds_data2']=bed_data_num
beds=[]
baths=[]
for i in df['beds_data2']:
    beds.append(i[0])
    baths.append(i[1])
df['beds']=beds
df['baths']=baths
df['beds'] = df['beds'].astype(int)
df['baths'] = df['baths'].astype(int)

#CLEAN ADDRESS
df['address'] = df['address'].astype('str')
df['address'] = df['address'].replace('<address class="list-card-addr">', ' ', regex=True)
df['address'] = df['address'].replace('</address>', ' ', regex=True)

#CLEAN PRICE
df['price']=df['price'].astype(str)
df['price'] = df['price'].replace('<div class="list-card-price">', ' ', regex=True)
df['price'] = df['price'].replace('</div>', ' ', regex=True)
df['price'] = df['price'].str.replace(',', '',regex=True)
df['price'] = df['price'].str.replace('$', '',regex=True)
df['price'] = df['price'].str.replace('+', '',regex=True)
df['price'] = df['price'].astype(int)


#CLEAN HEADER 

df['header'] = df['header'].astype(str)
df['header'] = df['header'].replace('<div class="list-card-variable-text list-card-img-overlay">', ' ', regex=True)
df['header'] = df['header'].replace('</div>', ' ', regex=True)

#CREATE 'DAYS ON THE MARKET' CATEGORY FROM THE HEADER, NAN FOR OTHER INFO
days_on=[]
for i in df['header']:
    if 'on Zillow' in i:
        days_on.append(re.findall(r'\d+', i)[0])
    elif 'hours ago' in i:
        days_on.append(1)
    else:
        days_on.append(np.nan)
df['days_on']=days_on  
#df['days_on'] = df['days_on'].astype(str)


#ADD TOWNS COLUMN
towns2=[]
for i in df['address']:
    towns2.append(i.split(',')[-2].rstrip(','))
df['town']=towns2


df2=df[['address','price','beds','baths','sqft', 'town', 'links','days_on','header']]#,'header' , 'days_on'
df3=df2.drop_duplicates(subset ="address", inplace = False)


In [466]:
df3.head(5)
#df3.shape

,address,price,beds,baths,sqft,town,links,days_on,header
0,"86 Rockland Ave, Larchmont, NY 10538",699000,3,3,1569,Larchmont,https://www.zillow.com/homedetails/86-Rockland...,5,5 days on Zillow
1,"1 Windsor St, Larchmont, NY 10538",1025000,4,3,2061,Larchmont,https://www.zillow.com/homedetails/1-Windsor-S...,7,7 days on Zillow
2,"62 Sherwood Dr, Larchmont, NY 10538",579000,3,2,1103,Larchmont,https://www.zillow.com/homedetails/62-Sherwood...,211,211 days on Zillow
3,"538 Weaver St, Larchmont, NY 10538",1995000,5,5,4360,Larchmont,https://www.zillow.com/homedetails/538-Weaver-...,NaN,Open: Sun. 1-3pm
4,"24 Homer Ave, Larchmont, NY 10538",1429000,4,4,2490,Larchmont,https://www.zillow.com/homedetails/24-Homer-Av...,6,6 days on Zillow


In [374]:
df3[50:100]

#df3=df3[~df3.address.str.contains("Modern Deluxe Plan")]
df3.shape


(215, 9)

In [467]:
#ZIPCODES
zipcode_listing=[]
zipcode_sold=[]
for i in df3['address']:
    zipcode_listing.extend(re.findall(r'\d+', i[-6:]))
for i in df_sold3['address']:
    zipcode_sold.extend(re.findall(r'\d+', i[-6:]))
df3['zipcode']=zipcode_listing
df_sold3['zipcode']=zipcode_sold
df_sold3['zipcode']=df_sold3['zipcode']

df3=df3[~df3.zipcode.str.contains("06904")]
df_sold3=df_sold3[~df_sold3.zipcode.str.contains("06904")]

df_sold3.head(5)

unique_zipcodes_listing=df3.zipcode.unique()
unique_zipcodes_sold=df_sold3.zipcode.unique()
#unique_zipcodes_listing=unique_zipcodes_listing[-1:5]
#unique_zipcodes_listing = ['06902', '06905', '06903', '06907', '06906', '06831']
unique_zipcodes_listing


'''
zipmask=(df3['zipcode']=='06904')
zipmask1=(df_sold3['zipcode']=='06904')
dropme6=df_sold3[zipmask1].index
dropme7=df3[zipmask].index
#df_sold3=df_sold3.copy()
df_sold3.drop(dropme6,inplace=True)
df3.drop(dropme7,inplace=True)
'''

"\nzipmask=(df3['zipcode']=='06904')\nzipmask1=(df_sold3['zipcode']=='06904')\ndropme6=df_sold3[zipmask1].index\ndropme7=df3[zipmask].index\n#df_sold3=df_sold3.copy()\ndf_sold3.drop(dropme6,inplace=True)\ndf3.drop(dropme7,inplace=True)\n"

## Webscrape ZIPMAPSDATA and merge with Zillow data

In [468]:
#WEBSCRAPE ZIPDATAMAPS

chromedriver = "/Applications/chromedriver" 
os.environ["webdriver.chrome.driver"] = chromedriver
link='https://www.zipdatamaps.com/'

full_zip_dict_web={}

for i in unique_zipcodes_listing:
    query = i 
    search=link+query
    page=requests.get(search).text
    soup=BeautifulSoup(page,'lxml')
    table=soup.find('table')
    rows = [row for row in table.find_all('tr')]
    
    avg_gross_household=[i.text for i in (rows[10].find_all('td'))]
    med_household = [i.text for i in (rows[9].find_all('td'))]
    school_test=[i.text for i in (rows[11].find_all('td'))]
    avg_commute_time=[i.text for i in (rows[12].find_all('td'))]
    full_zip_dict_web[i]=[avg_gross_household, med_household, school_test, avg_commute_time]
   
##    

query = '06901'
search=link+query
page=requests.get(search).text
soup=BeautifulSoup(page,'lxml')
table=soup.find('table')
rows = [row for row in table.find_all('tr')]
    
avg_gross_household2=[i.text for i in (rows[9].find_all('td'))]
med_household2 = [i.text for i in (rows[8].find_all('td'))]
school_test2=[i.text for i in (rows[10].find_all('td'))]
avg_commute_time2=[i.text for i in (rows[11].find_all('td'))]
full_zip_dict_web[query]=[avg_gross_household2, med_household2, school_test2, avg_commute_time2]  
 
##    
    
#greenwich ordering is 10,9,11,12
#scarsdale 10583 ordering is 12,11,13,14
    
#CLEAN AND APPLY ZIPDATAMAPS DATA

#SPLIT WEBSCRAPPED DICT INTO INDIVIDUAL ATTRIBUTES, ADD TO DATA FRAMES, CONVERT SCHOOL TO NUMBER
avg_gross={}
med_house={}
school_test={}
avg_commute={}
for i in full_zip_dict_web:
    avg_gross[i]=full_zip_dict_web[i][0][1]
    med_house[i]=full_zip_dict_web[i][1][1]
    school_test[i]=full_zip_dict_web[i][2][1]
    avg_commute[i]=full_zip_dict_web[i][3][1]

#print(avg_commute)    


school_dict={'Poor':1, 'Below Average': 2, 'Average': 3, 'Above Average': 4, 'Excellent':5}

df3["Avg_Commute"]=df3["zipcode"].map(avg_commute)   # -> do we need avg commute if adding lat/long?
df3["School_Performance"]=df3["zipcode"].map(school_test)
df3["Avg_Gross_Income"]=df3["zipcode"].map(avg_gross)
df3["Median_Household_Income"]=df3["zipcode"].map(med_house)
df3['School_Perforamnce_Num']=df3['School_Performance'].map(school_dict)

df_sold3["Avg_Commute"]=df_sold3["zipcode"].map(avg_commute)
df_sold3["School_Performance"]=df_sold3["zipcode"].map(school_test)
df_sold3["Avg_Gross_Income"]=df_sold3["zipcode"].map(avg_gross)
df_sold3["Median_Household_Income"]=df_sold3["zipcode"].map(med_house)
df_sold3['School_Perforamnce_Num']=df_sold3['School_Performance'].map(school_dict)

df3['Avg_Gross_Income'] = df3['Avg_Gross_Income'].str.replace('$', '',regex=True).astype(int)
df3['Median_Household_Income'] = df3['Median_Household_Income'].str.replace('$', '',regex=True).astype(int)
df_sold3['Avg_Gross_Income'] = df_sold3['Avg_Gross_Income'].str.replace('$', '',regex=True)
df_sold3['Median_Household_Income'] = df_sold3['Median_Household_Income'].str.replace('$', '',regex=True)
df_sold3['Avg_Commute'] = df_sold3['Avg_Commute'].str.replace('Minutes', '',regex=True).astype(float)
df3['Avg_Commute'] = df3['Avg_Commute'].str.replace('Minutes', '',regex=True).astype(float)



print(full_zip_dict_web)

{'10538': [['Average Adjusted Gross Income', '$460420'], ['Median Household Income', '$156105'], ['School Test Performance:', 'Excellent'], ['Average Commute Time', '38.5 Minutes']], '06901': [['Average Adjusted Gross Income', '$111330'], ['Median Household Income', '$48583'], ['School Test Performance:', 'Poor'], ['Average Commute Time', '25.3 Minutes']]}


## Use Geopy to translate addresses into lattitute and longitude, measure distance from midtown

In [ ]:
######
###### DONT RUN THIS EVERY TIME IT TAKES FOREVER##########
######
######
from geopy.extra.rate_limiter import RateLimiter
locator = Nominatim(user_agent='myGeocoder')
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
df3['location']=df3['address'].apply(geocode)
df_sold3['location']=df_sold3['address'].apply(geocode)


In [ ]:
df_sold3.tail(10)

In [ ]:
df3['point'] = df3['location'].apply(lambda loc: tuple(loc.point) if loc else None)
df_sold3['point'] = df_sold3['location'].apply(lambda loc: tuple(loc.point) if loc else None)
df3[['latitude', 'longitude','alt']] = pd.DataFrame(df3['point'].tolist(), index=df3.index)
df_sold3[['latitude,longitude,alt']] = pd.DataFrame(df_sold3['point'].tolist(), index=df_sold3.index)

#df_sold3[['latitude', 'longitude','alt']] = pd.DataFrame(df_sold3['point'].tolist(), index=df_sold3.index)
df_sold3.head(5)
df3.head(5)
df3['empire_lat'] = 40.748174
df3['empire_lon'] = -73.984776
df_sold3['empire_lat'] = 40.748174
df_sold3['empire_lon'] = -73.984776
df_sold3['listing_type']='S'
df3['listing_type']='L'

def distance(row): 
    address1 = (row['latitude'], row['longitude']) 
    address2 = (row['empire_lat'], row['empire_lon'])
         
    try: 
        return (geodesic(address1, address2).miles)
    except:
        return np.nan
    
df3['dist_downtown_nyc']=df3.apply(lambda row: distance(row), axis = 1 )
df_sold3['dist_downtown_nyc']=df_sold3.apply(lambda row: distance(row), axis = 1 )


#print(pd.DataFrame(df_sold3['point'].tolist(), index=df_sold3.index)[250:300])

In [ ]:
df3.head(5)

In [328]:
state1=[]
state2=[]
for i in df3['address']: 
    state1.append(i[-9:-6])
for i in df_sold3['address']: 
    state2.append(i[-9:-6])    
df3['state']=state1
df_sold3['state']=state2

mask1 = (df3['dist_downtown_nyc']<5) | (df3['dist_downtown_nyc']>80)
mask2 = (df_sold3['dist_downtown_nyc']<5) | (df_sold3['dist_downtown_nyc']>80)

dropme1=df3[mask1].index
df4=df3.copy()
df4.drop(dropme1,inplace=True)

dropme2=df_sold3[mask2].index
df_sold4=df_sold3.copy()
df_sold4.drop(dropme2,inplace=True)
df4.head(5)

,address,price,beds,baths,sqft,town,links,days_on,header,zipcode,...,location,point,latitude,longitude,alt,empire_lat,empire_lon,listing_type,dist_downtown_nyc,state
0,"81 Holmes Ave, Darien, CT 06820",995000,3,2,1554,Darien,https://www.zillow.com/homedetails/81-Holmes-A...,5,5 days on Zillow,06820,...,"(81, Holmes Avenue, Darien, Fairfield County, ...","(41.0711554, -73.5083455, 0.0)",41.071155,-73.508346,0.0,40.748174,-73.984776,L,33.448239,CT
1,"29 Brush Island Rd, Darien, CT 06820",6850000,5,6,5614,Darien,https://www.zillow.com/homedetails/29-Brush-Is...,NaN,"<svg class=""icon-3d-home"" viewbox=""0 0 21.46 ...",06820,...,"(29, Brush Island Road, Darien, Fairfield Coun...","(41.0501697, -73.49465, 0.0)",41.050170,-73.494650,0.0,40.748174,-73.984776,L,33.057800,CT
2,"15 Walmsley Rd, Darien, CT 06820",1895000,5,6,4400,Darien,https://www.zillow.com/homedetails/15-Walmsley...,10,10 days on Zillow,06820,...,"(15, Walmsley Road, Darien, Fairfield County, ...","(41.0659437, -73.4981228, 0.0)",41.065944,-73.498123,0.0,40.748174,-73.984776,L,33.614221,CT
3,"23 Shields Rd, Darien, CT 06820",1699000,4,4,4195,Darien,https://www.zillow.com/homedetails/23-Shields-...,NaN,"<span class=""price-reduction"">Price <!-- -->c...",06820,...,"(23, Shields Road, Noroton Heights, Darien, Fa...","(41.0760122, -73.4901196, 0.0)",41.076012,-73.490120,0.0,40.748174,-73.984776,L,34.384466,CT
4,"129 Nearwater Ln, Darien, CT 06820",4395000,6,6,5813,Darien,https://www.zillow.com/homedetails/129-Nearwat...,NaN,"<svg class=""icon-play"" focusable=""false"" view...",06820,...,"(129, Nearwater Lane, Noroton, Darien, Fairfie...","(41.0509648, -73.489476, 0.0)",41.050965,-73.489476,0.0,40.748174,-73.984776,L,33.302809,CT


In [329]:
print(df3.shape)
print(df_sold3.shape)

(39, 25)
(345, 25)


In [330]:
print(df4.shape)
print(df_sold4.shape)

(39, 25)
(345, 25)


In [294]:
df4.head(5)

,address,price,beds,baths,sqft,town,links,days_on,header,zipcode,...,location,point,latitude,longitude,alt,empire_lat,empire_lon,listing_type,dist_downtown_nyc,state
0,"14 White Plains Rd, Bronxville, NY 10708",725000,4,3,3103,Bronxville,https://www.zillow.com/homedetails/14-White-Pl...,NaN,"<svg class=""icon-3d-home"" viewbox=""0 0 21.46 ...",10708,...,"(14, White Plains Road, Tuckahoe, Town of East...","(40.9319261, -73.8184097, 0.0)",40.931926,-73.818410,0.0,40.748174,-73.984776,L,15.387779,NY
1,"16 Chester Pl, Bronxville, NY 10708",949000,5,3,2968,Bronxville,https://www.zillow.com/homedetails/16-Chester-...,NaN,"<svg class=""icon-play"" focusable=""false"" view...",10708,...,"(16, Chester Place, Eastchester, Town of Eastc...","(40.9297656, -73.8133266, 0.0)",40.929766,-73.813327,0.0,40.748174,-73.984776,L,15.418941,NY
2,"120 Oregon Ave APT D, Bronxville, NY 10708",469000,3,2,1300,Bronxville,https://www.zillow.com/homedetails/120-Oregon-...,NaN,"<span class=""price-reduction"">Price <!-- -->c...",10708,...,None,None,NaN,NaN,NaN,40.748174,-73.984776,L,NaN,NY
3,"145 Elmsmere Rd, Bronxville, NY 10708",1499000,5,3,3300,Bronxville,https://www.zillow.com/homedetails/145-Elmsmer...,21,21 days on Zillow,10708,...,"(145, Elmsmere Road, Elmsmere, Eastchester, Mo...","(40.9309706, -73.8180703, 0.0)",40.930971,-73.818070,0.0,40.748174,-73.984776,L,15.343650,NY
4,"4 Fordal Rd, Bronxville, NY 10708",7750000,6,8,8966,Bronxville,https://www.zillow.com/homedetails/4-Fordal-Rd...,84,84 days on Zillow,10708,...,"(4, Fordal Road, Bronxville, Town of Eastchest...","(40.9361609, -73.8176066, 0.0)",40.936161,-73.817607,0.0,40.748174,-73.984776,L,15.652808,NY


## Combine listed and sold data, pickle

In [331]:
darien_df1=pd.concat([df4, df_sold4], axis=0, ignore_index=True)

In [ ]:
bronxville_df1.head(10)

In [77]:
import pickle

In [332]:
picklefile=open('df_darien','wb')
pickle.dump(darien_df1,picklefile)

In [333]:
picklefile.close()

In [209]:
greenwich_df1.shape

(436, 26)

In [210]:
mamaroneck_df1.shape

(375, 26)

In [212]:
scarsdale_df1.shape

(567, 26)

In [224]:
rye_df1.describe()

,beds,baths,sqft,Avg_Commute,School_Perforamnce_Num,latitude,longitude,alt,empire_lat,empire_lon,dist_downtown_nyc,clean_price
count,444.000000,444.000000,444.000000,4.430000e+02,443.0,404.000000,404.000000,404.0,4.440000e+02,444.000000,404.000000,3.990000e+02
mean,4.502252,4.252252,3781.993243,3.520000e+01,5.0,40.972985,-73.690710,0.0,4.074817e+01,-73.984776,21.878294,2.036025e+06
std,1.329470,1.896741,2063.402221,7.113461e-15,0.0,0.015380,0.012210,0.0,7.113443e-15,0.000000,0.929036,1.446391e+06
min,2.000000,1.000000,1000.000000,3.520000e+01,5.0,40.945496,-73.722679,0.0,4.074817e+01,-73.984776,19.637144,2.280000e+05
25%,4.000000,3.000000,2246.000000,3.520000e+01,5.0,40.961626,-73.697709,0.0,4.074817e+01,-73.984776,21.170386,1.095000e+06
50%,5.000000,4.000000,3390.500000,3.520000e+01,5.0,40.969671,-73.688211,0.0,4.074817e+01,-73.984776,21.936129,1.730000e+06
75%,5.000000,5.000000,4713.000000,3.520000e+01,5.0,40.983122,-73.682925,0.0,4.074817e+01,-73.984776,22.613323,2.490000e+06
max,9.000000,12.000000,13658.000000,3.520000e+01,5.0,41.036466,-73.658183,0.0,4.074817e+01,-73.984776,25.507443,1.300000e+07
